In [1]:
import pandas as pd
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\sayan\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\sayan\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gazetteers.zip.
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\sayan\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\sayan\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gutenberg.zip.
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\sayan\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     C:\Users\sayan\AppData\Roaming\nl

True

In [2]:
import requests
url = 'https://raw.githubusercontent.com/SenticNet/personality-detection/master/essays.csv'
res = requests.get(url, allow_redirects=True)
with open('essays.csv','wb') as file:
    file.write(res.content)

In [6]:
df = pd.read_csv("essays.csv", encoding='cp1252')
df

,#AUTHID,TEXT,cEXT,cNEU,cAGR,cCON,cOPN
0,1997_504851.txt,"Well, right now I just woke up from a mid-day ...",n,y,y,n,y
1,1997_605191.txt,"Well, here we go with the stream of consciousn...",n,n,y,n,n
2,1997_687252.txt,An open keyboard and buttons to push. The thin...,n,y,n,y,y
3,1997_568848.txt,I can't believe it! It's really happening! M...,y,n,y,y,n
4,1997_688160.txt,"Well, here I go with the good old stream of co...",y,n,y,n,y
...,...,...,...,...,...,...,...
2462,2004_493.txt,I'm home. wanted to go to bed but remembe...,n,y,n,y,n
2463,2004_494.txt,Stream of consiousnesssskdj. How do you s...,y,y,n,n,y
2464,2004_497.txt,"It is Wednesday, December 8th and a lot has be...",n,n,y,n,n
2465,2004_498.txt,"Man this week has been hellish. Anyways, now i...",n,y,n,n,y


In [9]:
df.iloc[1016]


#AUTHID                                      1999_973407.txt
TEXT       For the past few days and now I have been thin...
cEXT                                                       y
cNEU                                                       n
cAGR                                                       y
cCON                                                       n
cOPN                                                       n
Name: 1016, dtype: object

In [10]:
letterToNumber = {
    "y":1,
    "n":0
}
try:
    for i,j in df.iterrows():
        if j["cEXT"] in letterToNumber.keys() or j["cNEU"] in letterToNumber.keys() or j["cAGR"] in letterToNumber.keys() or j["cOPN"] in letterToNumber.keys() or  j["cCON"] in letterToNumber.keys():
            j["cEXT"]=letterToNumber[j["cEXT"]]
            j["cNEU"]=letterToNumber[j["cNEU"]]
            j["cAGR"]=letterToNumber[j["cAGR"]]
            j["cOPN"]=letterToNumber[j["cOPN"]]
            j["cCON"]=letterToNumber[j["cCON"]]
except:
    print(i)

In [11]:
emoLex = pd.read_csv('Emotion_Lexicon.csv')
count = 0
emoWords=[]

for word in emoLex["Words"]:
  if emoLex["Charged"][count]==1:
    emoWords.append(word)

emoWordSet = set(emoWords)

emoLex.head()

,Words,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,Charged
0,march,0,0,0,0,0,0,1,0,0,0,1
1,august,0,0,0,0,0,0,1,0,0,0,1
2,ago,0,0,0,0,0,0,0,0,0,0,0
3,mar,0,0,0,0,0,1,0,0,0,0,1
4,vie,0,0,0,0,0,0,0,0,0,0,0


In [12]:
#text cleaning and tokenizing pipeline
import string
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

df['DOC']=''

all_words=[]


for i,j in df.iterrows():
  doc = []
  words = word_tokenize(j["TEXT"])
  wordSet = set(words)
  #check if it has any charged wordswembeddings
  if(emoWordSet & wordSet):
    #convert to lowercase
    words = [word.lower() for word in words]
    #remove punctuation from each word
    table = str.maketrans('','',string.punctuation)
    words = [word.translate(table) for word in words]
    #remove anything that isn't alpha
    words = [word for word in words if word.isalpha()]
    #remove stop words
    words = [word for word in words if not word in stop_words]
    for word in words:
      all_words.append(word)
      doc.append(word)
  j["DOC"]=doc
    
df["DOC"]

0       [well, right, woke, midday, nap, sort, weird, ...
1       [well, go, stream, consciousness, essay, used,...
2       [open, keyboard, buttons, push, thing, finally...
3       [ca, nt, believe, really, happening, pulse, ra...
4       [well, go, good, old, stream, consciousness, a...
                              ...                        
2462    [home, wanted, go, bed, remembered, psychology...
2463    [stream, consiousnesssskdj, spell, fuck, know,...
2464    [wednesday, december, lot, going, semester, tr...
2465    [man, week, hellish, anyways, time, minute, wr...
2466    [gotten, phone, brady, trying, decide, exacly,...
Name: DOC, Length: 2467, dtype: object

In [13]:
df["cEXT"]

0       0
1       0
2       0
3       1
4       1
       ..
2462    0
2463    1
2464    0
2465    0
2466    0
Name: cEXT, Length: 2467, dtype: object

In [14]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


max_len = 200


tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_words)

df["PADDED"]=''
sequence = tokenizer.texts_to_sequences(df["DOC"])
padded = pad_sequences(sequences=sequence,maxlen=max_len,padding='post')

for i,j in df.iterrows():
    if i<=1015:
        j["PADDED"]=padded[i]
    else:
        j["PADDED"]=padded[i-1]



In [ ]:
df.to_pickle('/content/drive/My Drive/NLP/Cleaned')
np.save('/content/drive/My Drive/NLP/word_index',tokenizer.word_index,allow_pickle=True)
np.save('/content/drive/My Drive/NLP/allwords',all_words,allow_pickle=True)
#df = pd.read_pickle('/content/drive/My Drive/NLP/Cleaned')

df["PADDED"]

0       [342, 20, 1, 690, 908, 7, 420, 4944, 283, 37, ...
1       [94, 317, 464, 32, 786, 199, 106, 28, 16, 1, 5...
2       [6248, 400, 286, 10, 30, 751, 11103, 231, 4120...
3       [20, 1, 135, 3, 1025, 7852, 4951, 2, 292, 2, 3...
4       [1158, 9, 52, 14, 562, 143, 6, 331, 1476, 30, ...
                              ...                        
2462    [37, 168, 10, 288, 1395, 210, 163, 641, 737, 9...
2463    [9, 30147, 30148, 498, 30149, 30150, 219, 498,...
2464    [1244, 3314, 32, 9, 277, 125, 275, 277, 644, 5...
2465    [45, 775, 29, 6414, 88, 23, 51, 2232, 1280, 39...
2466    [7765, 53, 2202, 101, 48, 30180, 51, 48, 13200...
Name: PADDED, Length: 2466, dtype: object

In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

In [21]:
import os
if os.name != 'nt':
    !unzip glove*.zip
else:
    print("Unzip the file manually on windows")

Unzip the file manually on windows


In [33]:
embedding_index = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:],dtype='float32')
  embedding_index[word]=coefs
f.close()
len(embedding_index)
len(coefs)

100

In [35]:
word_index = tokenizer.word_index
vocab = len(word_index)+1
EMBEDDING_DIM = 100

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [36]:
from sklearn.model_selection import train_test_split

X= padded
y=df["cEXT"]
X_train,X_test,y_train,y_test =  train_test_split(X,y,test_size=0.33,random_state=42)

print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(1652, 200) (815, 200) (1652,) (815,)


In [37]:
df

,#AUTHID,TEXT,cEXT,cNEU,cAGR,cCON,cOPN,DOC,PADDED
0,1997_504851.txt,"Well, right now I just woke up from a mid-day ...",0,1,1,0,1,"[well, right, woke, midday, nap, sort, weird, ...","[342, 20, 1, 690, 902, 7, 420, 4946, 283, 37, ..."
1,1997_605191.txt,"Well, here we go with the stream of consciousn...",0,0,1,0,0,"[well, go, stream, consciousness, essay, used,...","[94, 317, 464, 32, 786, 199, 106, 28, 16, 1, 5..."
2,1997_687252.txt,An open keyboard and buttons to push. The thin...,0,1,0,1,1,"[open, keyboard, buttons, push, thing, finally...","[6250, 400, 286, 10, 30, 751, 11112, 231, 4123..."
3,1997_568848.txt,I can't believe it! It's really happening! M...,1,0,1,1,0,"[ca, nt, believe, really, happening, pulse, ra...","[20, 1, 135, 3, 1025, 7854, 4953, 2, 292, 2, 3..."
4,1997_688160.txt,"Well, here I go with the good old stream of co...",1,0,1,0,1,"[well, go, good, old, stream, consciousness, a...","[1160, 9, 52, 14, 563, 143, 6, 331, 1476, 30, ..."
...,...,...,...,...,...,...,...,...,...
2462,2004_493.txt,I'm home. wanted to go to bed but remembe...,0,1,0,1,0,"[home, wanted, go, bed, remembered, psychology...","[1819, 1286, 3099, 869, 412, 178, 1668, 2569, ..."
2463,2004_494.txt,Stream of consiousnesssskdj. How do you s...,1,1,0,0,1,"[stream, consiousnesssskdj, spell, fuck, know,...","[37, 168, 10, 288, 1395, 210, 163, 641, 737, 9..."
2464,2004_497.txt,"It is Wednesday, December 8th and a lot has be...",0,0,1,0,0,"[wednesday, december, lot, going, semester, tr...","[9, 30181, 30182, 498, 30183, 30184, 219, 498,..."
2465,2004_498.txt,"Man this week has been hellish. Anyways, now i...",0,1,0,0,1,"[man, week, hellish, anyways, time, minute, wr...","[1244, 3317, 32, 9, 277, 125, 275, 277, 644, 5..."


In [38]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train,num_classes=2)
y_test = np_utils.to_categorical(y_test,num_classes=2)

In [39]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv2D,Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from keras import regularizers

def define_model(length,vocab):
  #channel for 1-gram
  input1 = Input(shape=(length,))
  embed1=Embedding(vocab,EMBEDDING_DIM,weights=[embedding_matrix],input_length=length,trainable=False)(input1)
  conv1 = Conv1D(filters=32,kernel_size=1,activation='relu')(embed1)
  drop1=Dropout(0.5)(conv1)
  pool1=MaxPooling1D(pool_size=2)(drop1)
  flat1=Flatten()(pool1)

  #Channel for 2-gram
  input2 = Input(shape=(length,))
  embed2=Embedding(vocab,EMBEDDING_DIM,weights=[embedding_matrix],input_length=length,trainable=False)(input2)
  conv2 = Conv1D(filters=32,kernel_size=2,activation='relu')(embed2)
  drop2=Dropout(0.5)(conv2)
  pool2=MaxPooling1D(pool_size=2)(drop2)
  flat2=Flatten()(pool2)

  #channel for 3-gram
  input3 = Input(shape=(length,))
  embed3=Embedding(vocab,EMBEDDING_DIM,weights=[embedding_matrix],input_length=length,trainable=False)(input3)
  conv3 = Conv1D(filters=32,kernel_size=3,activation='relu')(embed3)
  drop3=Dropout(0.5)(conv3)
  pool3=MaxPooling1D(pool_size=2)(drop3)
  flat3=Flatten()(pool3)

  #merge
  merged = concatenate([flat1,flat2,flat3])

  #MLP
  dense1 = Dense(256, activation='relu')(merged)
  dense2 = Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.01))(dense1)
  drop = Dropout(0.3)(dense2)
  dense3 = Dense(64, activation='relu')(drop)
  outputs = Dense(2, activation='softmax')(dense3)
  model = Model(inputs=[input1, input2, input3], outputs=outputs)
  # compile
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  # summarize
  print(model.summary())
  return model

In [40]:
model = define_model(length=max_len,vocab=vocab)
model.fit([X_train,X_train,X_train],y_train,epochs=50,batch_size=16,verbose=1)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 200, 100)     3022200     input_1[0][0]                    
______________________________________________________________________________________________

In [41]:
loss, acc = model.evaluate([X_test,X_test,X_test],y_test, verbose=0)
print(acc)

0.5153374075889587
